In [1]:
## Project 2
## Srikanth Shastry
## Sheetal Kulkarni

In [2]:
import pandas as pd
import hashlib
pd.set_option('display.max_rows', None)
import time


In [3]:
def pre_process_data(df):
    # Process the value in the first column (index 0)
    df[0] = df[0].split(":")[1].strip()
    
    # Process the value in the second column (index 1)
    df[1] = df[1].split(":")[1].strip()
    
    # Process the value in the third column (index 2)
    # Check if the value can be split using ":" as the delimiter
    if len(df[2].split(":")) > 1:
        # Select the second part (index 1) of the split list
        # Remove the last character (semicolon) using [:-1]
        df[2] = df[2].split(":")[1][:-1].strip()
    else:
        # If the value cannot be split, assume no trailing semicolon
        # Strip the value of leading and trailing spaces
        df[2] = df[2][:-1].strip()
    
    return df

In [4]:
# Read "100k.txt" into a pandas dataframe
raw_df = pd.read_csv("100k.txt",
                     header=None,
                     sep="\t",
                     quotechar='"',
                     skipinitialspace=True,
                     names=['subject', 'property', 'object'])

# Apply the pre_process_data function to each row of the dataframe
raw_df = raw_df.apply(pre_process_data, axis=1)

In [5]:
# Create a dictionary of dataframes
dict_df = dict(tuple(raw_df.groupby('property', as_index=False)))

# Remove "property" column from every dataframe in the dictionary
for key in dict_df:
    dict_df[key] = dict_df[key].drop("property", axis=1)
    dict_df[key].reset_index(drop=True, inplace=True)

In [6]:
def sortMergeJoin(df1, key1, df2, key2):
    # Perform Sorting based on keys
    sort_df1_On_key1 = df1.sort_values(by=key1, ascending=True)
    sort_df2_On_key2 = df2.sort_values(by=key2, ascending=True)
    
    # Perform merge join
    results_df = pd.merge(sort_df1_On_key1, sort_df2_On_key2, left_on=key1, right_on=key2, how='inner')

    return results_df

In [7]:
# Sort Merge Join
start_time = time.time()

# Perform sort merge join between "follows" and "friendOf" dataframes
followsObj_friendOfSubj_smj_df = sortMergeJoin(
    dict_df["follows"], "object",
    dict_df["friendOf"], "subject"
)

# Perform sort merge join between "likes" and "hasReview" dataframes
likesObj_hasReviewSubj_smj_df = sortMergeJoin(
    dict_df["likes"], "object",
    dict_df["hasReview"], "subject"
)

# Perform sort merge join between "followsObj_friendOfSubj_smj_df" and "likesObj_hasReviewSubj_smj_df" dataframes
friendsOfObj_likesSubj_smj_df = sortMergeJoin(
    followsObj_friendOfSubj_smj_df, "object_y",
    likesObj_hasReviewSubj_smj_df, "subject_x"
)

# Extract the desired columns from the joined dataframe
sortMergeJoin_QueryResult_df = friendsOfObj_likesSubj_smj_df[["subject_x_x",
                                                               "object_x_x",
                                                               "object_y_x",
                                                               "object_x_y",
                                                               "object_y_y"]]

# Rename the columns of the result dataframe
sortMergeJoin_QueryResult_df.rename(columns={'subject_x_x': 'follows.subject',
                                             'object_x_x': 'follows.object',
                                             'object_y_x': 'friendOf.object',
                                             'object_x_y': 'likes.object',
                                             'object_y_y': 'hasReview.object'}, inplace=True)

end_time = time.time()

/tmp/ipykernel_181/71858540.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sortMergeJoin_QueryResult_df.rename(columns={'subject_x_x': 'follows.subject',


In [8]:
print("Time Complexity of Sort merge Join: %s seconds" %(end_time - start_time))

Time Complexity of Sort merge Join: 6.768456220626831 seconds


In [9]:
def hashJoin(df1, key1, df2, key2):
    
    # Apply hash function on key columns of df1 and df2
    df1["hash_key"] = df1[key1].apply(hash)
    df2["hash_key"] = df2[key2].apply(hash)
    
    # Partition df1 based on hash keys
    df1_partitions = dict(tuple(df1.groupby('hash_key')))
    
    # Reset the index of each partition
    for key in df1_partitions:
        df1_partitions[key].reset_index(drop=True, inplace=True)
    
    # Get the keys present in df1 partitions
    dictKeys = df1_partitions.keys()
    
    # Initialize an empty list to store the joined partitions
    resultAsList = []
    
    # Iterate over rows of df2
    for i in range(len(df2)):
        # Check if the hash key of the row is present in df1 partitions
        if df2.iloc[i]['hash_key'] in dictKeys:
            key = df2.iloc[i]['hash_key']
            joinRow = df2.iloc[[i]]
            # Perform left join between partition and joinRow based on hash key
            joinedRow = pd.merge(df1_partitions[key], joinRow, on='hash_key', how='left')
            resultAsList.append(joinedRow.to_dict('list'))
    
    # Convert the list of dictionaries to a list of dataframes
    listOfJoinedPartitions = []
    for dictionary in resultAsList:
        listOfJoinedPartitions.append(pd.DataFrame(dictionary))
    
    # Concatenate the dataframes in the list
    finalDf = pd.concat(listOfJoinedPartitions)
    
    # Drop the hash_key column from the final dataframe
    finalDf = finalDf.drop('hash_key', axis=1)
    
    return finalDf

In [10]:
#Hash Join

start_time = time.time()

# Perform hash join between "follows" and "friendOf" dataframes
followsObj_friendOfSubj_hj_df = hashJoin(
    dict_df['follows'], 'object',
    dict_df['friendOf'], 'subject'
)

# Perform hash join between "likes" and "hasReview" dataframes
likesObj_hasReviewSubj_hj_df = hashJoin(
    dict_df['likes'], 'object',
    dict_df['hasReview'], 'subject'
)

# Perform hash join between "followsObj_friendOfSubj_hj_df" and "likesObj_hasReviewSubj_hj_df" dataframes
friendsOfObj_likesSubj_hj_df = hashJoin(
    followsObj_friendOfSubj_hj_df, 'object_y',
    likesObj_hasReviewSubj_hj_df, 'subject_x'
)

# Extract the desired columns from the joined dataframe
hashJoin_QueryResult_df = friendsOfObj_likesSubj_hj_df[["subject_x_x",
                                                        "object_x_x",
                                                        "object_y_x",
                                                        "object_x_y",
                                                        "object_y_y"]]

# Rename the columns of the result dataframe
hashJoin_QueryResult_df.rename(columns={'subject_x_x': 'follows.subject',
                                        'object_x_x': 'follows.object',
                                        'object_y_x': 'friendOf.object',
                                        'object_x_y': 'likes.object',
                                        'object_y_y': 'hasReview.object'}, inplace=True)

end_time = time.time()

/tmp/ipykernel_181/1922242578.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashJoin_QueryResult_df.rename(columns={'subject_x_x': 'follows.subject',


In [11]:
print("Time Complexity of Hash Join: %s seconds" %(end_time - start_time))

Time Complexity of Hash Join: 204.03605151176453 seconds


In [12]:
# Improved Hash-join

def improvedHashJoin(df1, key1, df2, key2):
    # Apply hash function on key columns of df1 and df2
    df1["hash_key"] = df1[key1].apply(hash)
    df2["hash_key"] = df2[key2].apply(hash)
    
    # Partition df1 based on hash keys
    df1_partitions = dict(tuple(df1.groupby('hash_key')))
    
    # Reset the index of each partition in df1
    for key in df1_partitions:
        df1_partitions[key].reset_index(drop=True, inplace=True)
    
    # Partition df2 based on hash keys
    df2_partitions = dict(tuple(df2.groupby('hash_key')))
    
    # Reset the index of each partition in df2
    for key in df2_partitions:
        df2_partitions[key].reset_index(drop=True, inplace=True)
    
    # Initialize an empty list to store the joined partitions
    resultAsList = []
    
    # Perform join between partitions with matching hash keys
    for key_2 in df2_partitions:
        for key_1 in df1_partitions:
            if key_1 == key_2:
                joinedPartitions = pd.merge(df1_partitions[key_1], df2_partitions[key_2], on='hash_key', how='left')
                resultAsList.append(joinedPartitions.to_dict('list'))
    
    # Convert the list of dictionaries to a list of dataframes
    listOfJoinedPartitions = []
    for dictionary in resultAsList:
        listOfJoinedPartitions.append(pd.DataFrame(dictionary))
    
    # Concatenate the dataframes in the list
    finalDf = pd.concat(listOfJoinedPartitions)
    
    # Drop the hash_key column from the final dataframe
    finalDf = finalDf.drop('hash_key', axis=1)
    
    return finalDf

In [13]:
#Improved Hash Join Results

start_time = time.time()

# Perform improved hash join between "follows" and "friendOf" dataframes
followsObj_friendOfSubj_ihj_df = improvedHashJoin(
    dict_df['follows'], 'object',
    dict_df['friendOf'], 'subject'
)

# Perform improved hash join between "likes" and "hasReview" dataframes
likesObj_hasReviewSubj_ihj_df = improvedHashJoin(
    dict_df['likes'], 'object',
    dict_df['hasReview'], 'subject'
)

# Perform improved hash join between "followsObj_friendOfSubj_ihj_df" and "likesObj_hasReviewSubj_ihj_df" dataframes
friendsOfObj_likesSubj_ihj_df = improvedHashJoin(
    followsObj_friendOfSubj_ihj_df, 'object_y',
    likesObj_hasReviewSubj_ihj_df, 'subject_x'
)

# Extract the desired columns from the joined dataframe
improvedHashJoin_QueryResult_df = friendsOfObj_likesSubj_ihj_df[["subject_x_x",
                                                                "object_x_x",
                                                                "object_y_x",
                                                                "object_x_y",
                                                                "object_y_y"]]

# Rename the columns of the result dataframe
improvedHashJoin_QueryResult_df.rename(columns={'subject_x_x': 'follows.subject',
                                                'object_x_x': 'follows.object',
                                                'object_y_x': 'friendOf.object',
                                                'object_x_y': 'likes.object',
                                                'object_y_y': 'hasReview.object'}, inplace=True)

end_time = time.time()

/tmp/ipykernel_181/3201583677.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  improvedHashJoin_QueryResult_df.rename(columns={'subject_x_x': 'follows.subject',


In [14]:
print("Time Complexity of Improved Hash Join: %s seconds" %(end_time - start_time))

Time Complexity of Improved Hash Join: 65.34886622428894 seconds
